In [ ]:
import carla
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

class CarlaEnv(gym.Env):
    def __init__(self):
        super(CarlaEnv, self).__init__()

        # Connect to CARLA server
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(5.0)
        self.world = self.client.get_world()

        # Action and observation spaces
        self.action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 3), dtype=np.uint8)

        # Get vehicle blueprint
        blueprint_library = self.world.get_blueprint_library()
        vehicle_bp = blueprint_library.find('vehicle.tesla.model3')

        # Get a valid spawn point
        spawn_point = self.get_valid_spawn_point()
        self.vehicle = self.world.try_spawn_actor(vehicle_bp, spawn_point)
        if self.vehicle is None:
            raise RuntimeError("Spawn failed, no valid spawn point available.")

        # Set up camera
        camera_bp = blueprint_library.find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', '84')
        camera_bp.set_attribute('image_size_y', '84')
        camera_bp.set_attribute('fov', '110')
        camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
        self.camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=self.vehicle)

    def get_valid_spawn_point(self):
        spawn_points = self.world.get_map().get_spawn_points()
        actors = self.world.get_actors()
        for point in spawn_points:
            occupied = False
            for actor in actors:
                if actor.get_location().distance(point.location) < 2.0:  # Minimum distance threshold
                    occupied = True
                    break
            if not occupied:
                return point
        return spawn_points[0]

    def reset(self):
        self.vehicle.set_transform(self.get_valid_spawn_point())
        self.world.tick()
        return np.zeros((84, 84, 3), dtype=np.uint8)

    def step(self, action):
        # Apply action to the vehicle
        throttle = float(action[0])
        steer = float(action[1])
        brake = float(action[2])
        
        self.vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer, brake=brake))

        # Get next state
        image = self.get_camera_image()
        
        # Calculate reward
        reward = self.calculate_reward()

        # Check if the episode is done
        done = self.is_done()
    
        return image, reward, done, {}


    def close(self):
        self.camera.destroy()
        self.vehicle.destroy()

In [18]:
# env = DummyVecEnv([lambda: CarlaEnv()])

# model = PPO('CnnPolicy', env, verbose=1)
# model.learn(total_timesteps=10000)

# model.save("carla_ppo")


In [ ]:
env = DummyVecEnv([lambda: CarlaEnv()])
model = PPO('CnnPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

/home/kaustubh/miniforge3/envs/carla_sim/lib/python3.7/site-packages/stable_baselines3/common/vec_env/patch_gym.py:50: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  "You provided an OpenAI Gym environment. "


Using cuda device
Wrapping the env in a VecTransposeImage.
-----------------------------
| time/              |      |
|    fps             | 40   |
|    iterations      | 1    |
|    time_elapsed    | 51   |
|    total_timesteps | 2048 |
-----------------------------
--------------------------------------
| time/                   |          |
|    fps                  | 39       |
|    iterations           | 2        |
|    time_elapsed         | 102      |
|    total_timesteps      | 4096     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -2.84    |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 10       |
|    policy_gradient_loss | 0        |
|    std                  | 1        |
|    value_loss           | 0        |
-----------------------------

KeyboardInterrupt: 

: 